In [1]:
import os
currentDirectory = os.getcwd()
data_dir = currentDirectory[:-7] + '/input/'
print(data_dir)

from xmitgcm import open_mdsdataset
ds1 = open_mdsdataset(data_dir, geometry='cartesian', endian='<',prefix=['statevars'],iters=range(100*1860,170*1860,1860))

/scratch/jxchang/HighRes1/results/TideU008N0LinH200ho140Ah0200Cdqdt003/input/


In [2]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import cm
from matplotlib.colors import BoundaryNorm
import matplotlib.animation as animation
import math
%matplotlib inline

In [3]:
t = 0
h0 = 140.

f0 = 1.e-4
g = 9.8
u0=0.08

om=2*np.pi/12.4/3600 
alpha = 2e-4 

tR_fname="../indata/TRef.bin"
tRef = np.fromfile(tR_fname)
refSalt=35.
refTemp=tRef[0]
print('refTemp='+ str(refTemp))
print('tRef='+ str(tRef))

xmin = -4
xmax = 4
vmax = 3
vmin = -3
wmax = 0.2
wmin = -0.2
tmax = math.floor(np.amax(tRef))
tmin = math.floor(np.amin(tRef))
numcolt=21
numcolv=30

ttlen=len(ds1.time)
print('the length of time:' + str(ttlen) )
print(tmin)

refTemp=63.59230581861823
tRef=[63.59230582 40.69584406 33.47293155 31.54712706 29.26896757 28.65696899
 28.23721628 27.65390158 27.08300659 26.5118492  25.90295299 25.69407634
 25.29910279 24.81832859 24.41499174 24.26210069 24.21619203 24.14261024
 24.0996551  24.02495734 23.98958682 23.95165181 23.93411373 23.9224752
 23.91844136 23.91059116 23.87093593 23.86487752 23.86221099 23.85790713
 23.84947122 23.84359894 23.82719386 23.81078878 23.79438371 23.77797863
 23.76157355 23.74516847 23.72876339 23.71235831]
the length of time:70
23


In [4]:
xg=ds1.coords["XG"]
xc=ds1.coords["XC"]
yg=ds1.coords["YG"]
yc=ds1.coords["YC"]
z=ds1.coords["Z"]
time=ds1.coords['time'].values/np.timedelta64(1, 's')/3600
ds1=ds1.assign_coords(XC0 = (ds1.XC/1000.-ds1.XC.mean()/1000.))
ds1=ds1.assign_coords(XG0 = (ds1.XG-ds1.XC.mean())/1000.)
ds1=ds1.assign_coords(YC0 = (ds1.YC/1000.-ds1.YC.mean()/1000.))
ds1=ds1.assign_coords(YG0 = (ds1.YG-ds1.YC.mean())/1000.)
yc0 = (ds1.YC/1000.-ds1.YC.mean()/1000.)
xc0 = (ds1.XC/1000.-ds1.XC.mean()/1000.)

In [5]:
fig = plt.figure(figsize=(11,8))
grid = plt.GridSpec(2, 2, height_ratios=[0.6,2.5], width_ratios=[1,0.1],wspace=0.02,hspace=0.38)

<Figure size 792x576 with 0 Axes>

In [6]:
SMALL_SIZE = 17
MEDIUM_SIZE = 19
BIGGER_SIZE = 21

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

In [7]:
#plot bathymetry
ax1=fig.add_subplot(grid[0,:-1])
ax1.plot(time,np.zeros(ttlen),color='gray',linestyle='--')
ax1.plot(time,ds1.UVEL.isel(Z=0,YC=0,XG=0),label='U(t,0,0,0)')
ax1.plot(time,u0*np.cos(om*ds1.time.values/np.timedelta64(1, 's')),label='Uinput')
ax1.set_xlim(min(time), max(time))
ax1.set_ylim(-u0-0.05, u0+0.05)
point, = ax1.plot([time[0]],[ds1.UVEL.isel(time=t,YC=40,XG=0,Z=0).data], 'go', color='red',ms=10)
ax1.set_xlabel('time [hr]')
ax1.set_ylabel('U [m/s]')
ax1.set_title('Uinput vs U[t,0,0,0] ', loc='left')



Text(0.0, 1.0, 'Uinput vs U[t,0,0,0] ')

In [8]:
# plot temperature
ax2=fig.add_subplot(grid[1,0])
cmap = cm.Spectral_r
levels = np.linspace(23, 45, num=numcolt)
norm = BoundaryNorm(levels, ncolors=cmap.N, clip=True)
opts = {'cmap':cm.get_cmap(cmap, len(levels) - 1), 'norm': norm}

cset1 = ax2.contourf(ds1.XC0, ds1.YC0, ds1.THETA.isel(time=t,Z=6), levels, **opts)

cax = fig.add_subplot(grid[1,-1])
cax.set_visible(False)
fig.colorbar(cset1, ax=cax,extend='both')
#fig.colorbar(cset1, ax=axs,ticks=np.arange(tmin, tmax+0.1, 5))
ax2.set_xlabel('X [km]')
ax2.set_ylabel('Y [km]')
ax2.set_title('T [oC]', loc='left')
ax2.set(xlim=(xmin, xmax))
#ax2.plot(ds1['XC0'],-ds1.Depth.isel(YC=60),lw=1.5)

/home/jxchang/jupyter_py3.7/lib/python3.7/site-packages/ipykernel_launcher.py:12: MatplotlibDeprecationWarning: The 'extend' parameter to Colorbar has no effect because it is overridden by the mappable; it is deprecated since 3.3 and will be removed two minor releases later.
  if sys.path[0] == '':


[(-4.0, 4.0)]

In [9]:
from scipy.interpolate import griddata
print(xc.shape)
Xu,Yu=np.meshgrid(xg.values,yc.values)
Xv,Yv=np.meshgrid(xc.values,yg.values)
print(np.shape(Xu))
points_u = np.transpose([Xu.flatten(),Yu.flatten()])
points_v = np.transpose([Xv.flatten(),Yv.flatten()])
print(np.shape(points_u))
grid_x, grid_y = np.mgrid[36000:44000:250,0:3000:150]
grid_U = griddata(points_u,(ds1.UVEL.isel(time=t,Z=6).values).flatten(),(grid_x, grid_y))
grid_V = griddata(points_v,(ds1.VVEL.isel(time=t,Z=6).values).flatten(),(grid_x, grid_y))
grid_x0=grid_x/1000.-ds1.XC.values.mean()/1000.
grid_y0=grid_y/1000.-ds1.YC.values.mean()/1000.
print(grid_x0[0,:5])

(1440,)
(120, 1440)
(172800, 2)
[-4. -4. -4. -4. -4.]


In [10]:
quiv = ax2.quiver(grid_x0, grid_y0, grid_U, grid_V, color='k')
ax2.quiverkey(quiv,0.93,0.02,1,r'$1 \frac{m}{s}$', labelpos='E')
plt.show()

In [11]:
fig
print(ax2.collections)
print(np.size(ax2.collections))

[<matplotlib.collections.PathCollection object at 0x2b913ffd8590>, <matplotlib.collections.PathCollection object at 0x2b913ffd8890>, <matplotlib.collections.PathCollection object at 0x2b91479f6ad0>, <matplotlib.collections.PathCollection object at 0x2b9147a024d0>, <matplotlib.collections.PathCollection object at 0x2b9147a0dd10>, <matplotlib.collections.PathCollection object at 0x2b9147a1dcd0>, <matplotlib.collections.PathCollection object at 0x2b9147a28d10>, <matplotlib.collections.PathCollection object at 0x2b9147a2da90>, <matplotlib.collections.PathCollection object at 0x2b9147a35c10>, <matplotlib.collections.PathCollection object at 0x2b9147a1dd10>, <matplotlib.collections.PathCollection object at 0x2b9147a3a790>, <matplotlib.collections.PathCollection object at 0x2b9147a3ac50>, <matplotlib.collections.PathCollection object at 0x2b9147a42150>, <matplotlib.collections.PathCollection object at 0x2b9147a42550>, <matplotlib.collections.PathCollection object at 0x2b9147a42ad0>, <matplotl

In [12]:
tt: int=0
def updateData(tt):

    grid_U = griddata(points_u,(ds1.UVEL.isel(time=tt,Z=6).values).flatten(),(grid_x, grid_y))
    grid_V = griddata(points_v,(ds1.VVEL.isel(time=tt,Z=6).values).flatten(),(grid_x, grid_y))
    print(tt)
    print(np.amax(grid_U))
    print(np.amax(grid_V))
    updatetime = time[tt]
    ax2.collections[0:numcolt-1] = []
    ax2.contourf(ds1.XC0, ds1.YC0, ds1.THETA.isel(time=tt,Z=6), levels, **opts)
    print(np.size(ax2.collections))
    print(ax2.collections)
    ax2.collections.append(ax2.collections.pop(0))
    quiv.set_UVC(grid_U,grid_V)
    point.set_data(updatetime,ds1.UVEL.isel(time=tt,YC=60,XG=0,Z=0))
    ax1.set_title('t= %2.3fhr' % updatetime)
    fig1 = plt.gcf()
    fig.savefig('./figsMag2/summary_TUvec_z6_xy_t%s.png' %(tt+100))



simulation = animation.FuncAnimation(fig, updateData, blit=False, frames= 40 , interval=40, repeat=False)
plt.draw()
plt.show()
simulation.save(filename='summary_TUvec_z6_xy.gif', writer='imagemagick', fps=2)

<Figure size 432x288 with 0 Axes>

0
nan
0.5422157411634391
21
[<matplotlib.quiver.Quiver object at 0x2b9147a6f910>, <matplotlib.collections.PathCollection object at 0x2b9148ba6b90>, <matplotlib.collections.PathCollection object at 0x2b9148bb10d0>, <matplotlib.collections.PathCollection object at 0x2b914c0a4490>, <matplotlib.collections.PathCollection object at 0x2b914c0ade50>, <matplotlib.collections.PathCollection object at 0x2b914c0bb710>, <matplotlib.collections.PathCollection object at 0x2b914c0cb690>, <matplotlib.collections.PathCollection object at 0x2b914c0d66d0>, <matplotlib.collections.PathCollection object at 0x2b914c0e1450>, <matplotlib.collections.PathCollection object at 0x2b914c0e7590>, <matplotlib.collections.PathCollection object at 0x2b914c0cb6d0>, <matplotlib.collections.PathCollection object at 0x2b914c0ed150>, <matplotlib.collections.PathCollection object at 0x2b914c0ed610>, <matplotlib.collections.PathCollection object at 0x2b914c0edad0>, <matplotlib.collections.PathCollection object at 0x2b914c0ed

5
nan
0.3136905283803712
21
[<matplotlib.quiver.Quiver object at 0x2b9147a6f910>, <matplotlib.collections.PathCollection object at 0x2b9148d9c6d0>, <matplotlib.collections.PathCollection object at 0x2b9148c35110>, <matplotlib.collections.PathCollection object at 0x2b9148ea1790>, <matplotlib.collections.PathCollection object at 0x2b9148ea5350>, <matplotlib.collections.PathCollection object at 0x2b9148eb5690>, <matplotlib.collections.PathCollection object at 0x2b9148e006d0>, <matplotlib.collections.PathCollection object at 0x2b9148db2310>, <matplotlib.collections.PathCollection object at 0x2b9148e19d50>, <matplotlib.collections.PathCollection object at 0x2b9148d7ecd0>, <matplotlib.collections.PathCollection object at 0x2b9148e00e10>, <matplotlib.collections.PathCollection object at 0x2b9148e4eb10>, <matplotlib.collections.PathCollection object at 0x2b9148d5cf90>, <matplotlib.collections.PathCollection object at 0x2b9148ecad50>, <matplotlib.collections.PathCollection object at 0x2b9148eca

11
nan
0.3566197870393658
21
[<matplotlib.quiver.Quiver object at 0x2b9147a6f910>, <matplotlib.collections.PathCollection object at 0x2b9148d60b90>, <matplotlib.collections.PathCollection object at 0x2b9148c65810>, <matplotlib.collections.PathCollection object at 0x2b9148dc1750>, <matplotlib.collections.PathCollection object at 0x2b914c10f590>, <matplotlib.collections.PathCollection object at 0x2b9148e395d0>, <matplotlib.collections.PathCollection object at 0x2b9148e62650>, <matplotlib.collections.PathCollection object at 0x2b9148ec0f90>, <matplotlib.collections.PathCollection object at 0x2b9148ed8690>, <matplotlib.collections.PathCollection object at 0x2b9148e0bfd0>, <matplotlib.collections.PathCollection object at 0x2b9148e62850>, <matplotlib.collections.PathCollection object at 0x2b9148e01d50>, <matplotlib.collections.PathCollection object at 0x2b9148e01490>, <matplotlib.collections.PathCollection object at 0x2b9148ec61d0>, <matplotlib.collections.PathCollection object at 0x2b9148ea

17
nan
0.27728664189849245
21
[<matplotlib.quiver.Quiver object at 0x2b9147a6f910>, <matplotlib.collections.PathCollection object at 0x2b9148e95710>, <matplotlib.collections.PathCollection object at 0x2b9148e95a90>, <matplotlib.collections.PathCollection object at 0x2b9148e9df10>, <matplotlib.collections.PathCollection object at 0x2b9148ed8ad0>, <matplotlib.collections.PathCollection object at 0x2b9147aa3a10>, <matplotlib.collections.PathCollection object at 0x2b9148e4e390>, <matplotlib.collections.PathCollection object at 0x2b9148e62190>, <matplotlib.collections.PathCollection object at 0x2b9148db3e50>, <matplotlib.collections.PathCollection object at 0x2b9148e67a50>, <matplotlib.collections.PathCollection object at 0x2b9148e4e2d0>, <matplotlib.collections.PathCollection object at 0x2b9148e67c90>, <matplotlib.collections.PathCollection object at 0x2b9148e672d0>, <matplotlib.collections.PathCollection object at 0x2b9148e65350>, <matplotlib.collections.PathCollection object at 0x2b9148e

23
nan
0.2886716437185378
21
[<matplotlib.quiver.Quiver object at 0x2b9147a6f910>, <matplotlib.collections.PathCollection object at 0x2b9148d88710>, <matplotlib.collections.PathCollection object at 0x2b9148deb190>, <matplotlib.collections.PathCollection object at 0x2b9148ba2150>, <matplotlib.collections.PathCollection object at 0x2b9148ea6310>, <matplotlib.collections.PathCollection object at 0x2b9148e67190>, <matplotlib.collections.PathCollection object at 0x2b9148ead390>, <matplotlib.collections.PathCollection object at 0x2b9148c622d0>, <matplotlib.collections.PathCollection object at 0x2b9148b9fa50>, <matplotlib.collections.PathCollection object at 0x2b9148c60250>, <matplotlib.collections.PathCollection object at 0x2b9148ead4d0>, <matplotlib.collections.PathCollection object at 0x2b9148b7bcd0>, <matplotlib.collections.PathCollection object at 0x2b9148ed8450>, <matplotlib.collections.PathCollection object at 0x2b9148ed8f10>, <matplotlib.collections.PathCollection object at 0x2b9148ed

29
nan
0.29341609266058366
21
[<matplotlib.quiver.Quiver object at 0x2b9147a6f910>, <matplotlib.collections.PathCollection object at 0x2b9148c65890>, <matplotlib.collections.PathCollection object at 0x2b9148db3b90>, <matplotlib.collections.PathCollection object at 0x2b913ffc4fd0>, <matplotlib.collections.PathCollection object at 0x2b914c0a4250>, <matplotlib.collections.PathCollection object at 0x2b9148dc4090>, <matplotlib.collections.PathCollection object at 0x2b9148e9b7d0>, <matplotlib.collections.PathCollection object at 0x2b914c0ed590>, <matplotlib.collections.PathCollection object at 0x2b9148c60650>, <matplotlib.collections.PathCollection object at 0x2b914c0cba10>, <matplotlib.collections.PathCollection object at 0x2b9148e9b190>, <matplotlib.collections.PathCollection object at 0x2b9148d63610>, <matplotlib.collections.PathCollection object at 0x2b9148d63450>, <matplotlib.collections.PathCollection object at 0x2b9148ec0890>, <matplotlib.collections.PathCollection object at 0x2b9148d

35
nan
0.45828066431115044
21
[<matplotlib.quiver.Quiver object at 0x2b9147a6f910>, <matplotlib.collections.PathCollection object at 0x2b914c10fa90>, <matplotlib.collections.PathCollection object at 0x2b914c10f610>, <matplotlib.collections.PathCollection object at 0x2b914c0e1650>, <matplotlib.collections.PathCollection object at 0x2b9148eb5b10>, <matplotlib.collections.PathCollection object at 0x2b9148e2d590>, <matplotlib.collections.PathCollection object at 0x2b914c0edd10>, <matplotlib.collections.PathCollection object at 0x2b9148b8ffd0>, <matplotlib.collections.PathCollection object at 0x2b9148e9b390>, <matplotlib.collections.PathCollection object at 0x2b9148e46750>, <matplotlib.collections.PathCollection object at 0x2b914c0eddd0>, <matplotlib.collections.PathCollection object at 0x2b914c0adf10>, <matplotlib.collections.PathCollection object at 0x2b9148e44c90>, <matplotlib.collections.PathCollection object at 0x2b9148e44250>, <matplotlib.collections.PathCollection object at 0x2b9148e

In [13]:
from IPython.display import Image
from IPython.display import display
display(Image(url='./summary_TUvec_z6_xy.gif'))